## 과제. VGG19 구현하기

![VGG19](https://qcloud.coding.net/u/vincentqin/p/blogResource/git/raw/master/CNN-Architectures/89475867.png)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.autograd import Variable

from torchvision import transforms
from torchvision import datasets

batch_size = 4
learning_rate = 0.01
num_epochs = 5

transform = transforms.ToTensor()

# Data Loader
trainData = datasets.CIFAR10(root="./data",train=True,transform=transform,download=True)
trainLoader = DataLoader(trainData,batch_size=batch_size,shuffle=True)

testData = datasets.CIFAR10(root="./data", train=False, transform=transform)
testLoader = DataLoader(testData, batch_size=batch_size, shuffle=False)

In [ ]:
def conv(c_in, c_out, k_size, stride=1, pad=1, bn=True):
    """Custom convolutional layer for simplicity."""
    layers = []
    layers.append(nn.Conv2d(c_in, c_out, k_size, stride, pad, bias=False))
    if bn:
        layers.append(nn.BatchNorm2d(c_out))
    layers.append(nn.ReLU())
    return nn.Sequential(*layers)

In [ ]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()

        self.layer1 = nn.Sequential(
            conv(3, 64, 3),
            conv(64, 64, 3),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(
            conv(64, 128, 3),
            conv(128, 128, 3),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = nn.Sequential(
            conv(128, 256, 3),
            conv(256, 256, 3),
            conv(256, 256, 3),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer4 = nn.Sequential(
            conv(256, 512, 3),
            conv(512, 512, 3),
            conv(512, 512, 3),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer5 = nn.Sequential(
            conv(512, 512, 3),
            conv(512, 512, 3),
            conv(512, 512, 3),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.FC1 = nn.Sequential(
            nn.Linear(512, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU())

        self.FC2 = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU())

        self.FC3 = nn.Sequential(
            nn.Linear(4096, 10),
            nn.BatchNorm1d(10),
            nn.Softmax())

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)

        out = out.view(out.size(0), -1)

        out = self.FC1(out)
        out = self.FC2(out)
        out = self.FC3(out)

        return out

In [ ]:
vgg16 = VGG16()

# Loss Function
Loss_function = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(vgg16.parameters(), lr=learning_rate)


In [ ]:
# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainLoader):
        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = Loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('Epoch [%d/%d], Iter [%d/%d] Loss %.4f'
                  % (epoch+1, num_epochs, i, len(trainLoader), loss.item()))

In [ ]:
# Test the model
vgg16.eval()
correct = 0
total = 0

for images, labels in testLoader:
    images = Variable(images).cuda()
    outputs = vgg16(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

print('Test Accuracy : %d %%' % (100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model_nn.ckpt')